<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/17_Finans_i%C3%A7in_Python_Bollinger_Band.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Kütüphanelerin yüklenmesi
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener
!pip install backtesting

#kütüphanelerin çağırılması
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from backtesting import Backtest, Strategy
import numpy as np
#Tradingview Kütüphanesini açıyoruz.
tv = TvDatafeed()

# SMA Fonksiyonu
def sma(series, length):
    """
    Belirli bir serinin Basit Hareketli Ortalamasını (SMA) hesaplar.
    """
    return series.rolling(window=length).mean()

# Standart Sapma Fonksiyonu
def stdev(series, length):
    """
    Verilen bir pandas serisi için belirli bir pencere uzunluğuna göre hareketli standart sapmayı hesaplar.
    """
    # Hareketli pencere içinde belirlenen uzunluk (length) için ortalama hesapla
    avg = series.rolling(window=length).mean()

    # Hareketli pencere içinde her bir veri kümesi için kare sapmaların toplamını hesapla
    sumOfSquareDeviations = series.rolling(window=length).apply(
        lambda x: ((x - avg.loc[x.index[-1]]) ** 2).sum(),
        raw=False)

    # Kare sapmaların toplamını pencere uzunluğuna bölerek standart sapmayı hesapla
    deviation = np.sqrt(sumOfSquareDeviations / length)
    return deviation

#Bollinger Band Fonksiyonu
def bollinger_bands(series, length=20, std_multiplier=2):
    """
    Bollinger Bantlarını hesaplar.
    """
    middle_band = sma(series, length)
    std = stdev(series, length)
    print(std*std_multiplier)
    upper_band = middle_band + std * std_multiplier
    lower_band = middle_band - std * std_multiplier
    return lower_band, upper_band


# Strateji Simülatörümüzü çağırıyoruz.
class Strategy(Strategy):
    def init(self):
        pass
    def next(self):
        if self.data['Entry'] == True and not self.position:
            self.buy()
        elif self.data['Exit'] == True:
            self.position.close()

Hisse = "DOAS"
data = tv.get_hist(symbol=Hisse, exchange='BIST', interval=Interval.in_daily, n_bars=5000)
print(data)

data['BB_Lower'],data['BB_Upper'] = bollinger_bands(data['close'],20,2)
print(data)



data['Entry'] = (data['close'] > data['BB_Lower']) & (data['close'].shift(1) < data['BB_Lower'])
data['Exit'] = data['close'] > data['BB_Upper']


# Veri başlıklarımızı yeniden adlandırıyoruz.
data.rename(columns={'open': 'Open', 'high': 'High',
                     'low': 'Low',
                     'close': 'Close',
                     'volume': 'Volume'},
                     inplace=True)


# Backtest simulatörünü çalıştırma
bt = Backtest(data, Strategy, cash=100000, commission=0.002)
Stats = bt.run()
print(Stats)

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-n_e0_1ag
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-n_e0_1ag
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


                        symbol      open       high        low      close  \
datetime                                                                    
2004-09-07 06:00:00  BIST:DOAS    2.4624    2.48520    2.41680    2.43960   
2004-09-08 06:00:00  BIST:DOAS    2.4624    2.48520    2.39400    2.39400   
2004-09-09 06:00:00  BIST:DOAS    2.4168    2.43960    2.37120    2.41680   
2004-09-10 06:00:00  BIST:DOAS    2.4168    2.41680    2.34840    2.37120   
2004-09-13 06:00:00  BIST:DOAS    2.3484    2.37120    2.32560    2.37120   
...                        ...       ...        ...        ...        ...   
2024-07-31 06:00:00  BIST:DOAS  244.5000  245.80000  240.00000  240.50000   
2024-08-01 06:00:00  BIST:DOAS  242.5000  245.10001  242.00000  242.80000   
2024-08-02 06:00:00  BIST:DOAS  242.3000  242.50000  236.00000  237.89999   
2024-08-05 06:00:00  BIST:DOAS  229.7000  232.00000  216.60001  230.00000   
2024-08-06 06:00:00  BIST:DOAS  234.5000  240.30000  229.80000  231.70000   